This is an introduction to Tensorflow 2 with the Keras API. We will work with the MNIST dataset of hand-written digits and:
*   classify them using logistic regression and multi-layer dense networks
*   examine various network depths, activation functions and optimizers
*   introduce convolutional networks and the LeNet5 architecture

Import Tensorflow, load the MNIST dataset using Tensorflow built-in tools and display some characteristics of the data and target arrays:

In [ ]:
import tensorflow as tf

(DATA0, TARGET0), (DATA1, TARGET1) = tf.keras.datasets.mnist.load_data()

print(type(DATA0), DATA0.shape, DATA0.dtype, DATA0.min(), DATA0.max())
print(type(TARGET0), TARGET0.shape, TARGET0.dtype, TARGET0.min(), TARGET0.max())

print(type(DATA1), DATA1.shape, DATA1.dtype, DATA1.min(), DATA1.max())
print(type(TARGET1), TARGET1.shape, TARGET1.dtype, TARGET1.min(), TARGET1.max())

11501568/11490434 [==============================] - 0s 0us/step
<class 'numpy.ndarray'> (60000, 28, 28) uint8 0 255
<class 'numpy.ndarray'> (60000,) uint8 0 9
<class 'numpy.ndarray'> (10000, 28, 28) uint8 0 255
<class 'numpy.ndarray'> (10000,) uint8 0 9


These are ordinary numpy arrays. There are 60 000 samples in the training dataset and 10 000 in the validation dataset. The targets are integer numbers from 0 to 9 labeling the digits. The data are 28x28 grayscale images with values of type uint8, ranging from 0 to 255.

**Task.** Display the first image from the training dataset.

Dense networks take flat vectors of floating-point features and are best suited for inputs of the order of 1 rather than 255. So flatten the images, cast them to float32 and scale to the range from 0 to 1:

In [ ]:
DATA0 = DATA0.reshape(-1, 28 * 28).astype('float32') / 255.
DATA1 = DATA1.reshape(-1, 28 * 28).astype('float32') / 255.

Build the simplest Tensorflow model. This is a single-layer dense network with softmax activation, equivalent to logistic regression. It has 10 outputs corresponding to digits from 0 to 9:

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10),
    tf.keras.layers.Softmax()])

The activation functions, like softmax, sigmoid, relu, etc., can be treated as separate layers or equivalently added at the outputs of the dense layers:

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, activation = 'softmax')])

The model has not been trained yet, so it will give incorrect predictions. But anyway pass the entire validation dataset thereto and display the output:

In [ ]:
PROBABILITY1 = model(DATA1)
print(PROBABILITY1)

tf.Tensor(
[[0.11072053 0.07813346 0.09716906 ... 0.14569585 0.07165894 0.13054055]
 [0.03729106 0.08779248 0.06638742 ... 0.08476997 0.18627657 0.12411804]
 [0.07035261 0.10220047 0.09934175 ... 0.1036746  0.10641313 0.13612045]
 ...
 [0.05512704 0.16106337 0.11587422 ... 0.04533249 0.05776504 0.19524232]
 [0.06082381 0.10118923 0.13928978 ... 0.08741697 0.09241118 0.13897476]
 [0.03037542 0.06121332 0.22899926 ... 0.04853882 0.08292364 0.09486538]], shape=(10000, 10), dtype=float32)


The output is not a numpy array but a tf.Tensor, an internal format on which Tensorflow operates. Convert it back to a numpy array for further processing:

In [ ]:
PROBABILITY1 = model(DATA1).numpy()
print(PROBABILITY1)

[[0.11072053 0.07813346 0.09716906 ... 0.14569585 0.07165894 0.13054055]
 [0.03729106 0.08779248 0.06638742 ... 0.08476997 0.18627657 0.12411804]
 [0.07035261 0.10220047 0.09934175 ... 0.1036746  0.10641313 0.13612045]
 ...
 [0.05512704 0.16106337 0.11587422 ... 0.04533249 0.05776504 0.19524232]
 [0.06082381 0.10118923 0.13928978 ... 0.08741697 0.09241118 0.13897476]
 [0.03037542 0.06121332 0.22899926 ... 0.04853882 0.08292364 0.09486538]]


Each row of this array contains 10 probabilities of the sample representing digits from 0 to 9.

**Task.** Run the code again from where the model is constructed and inspect the resulting probabilities. Why are these probabilities different every time the code is run? How does this relate to scaling the inputs to the range from 0 to 1?

**Task.** Given the probabilities, calculate the digits to which the samples are classified.

**Task.** Compare the labels from the model with the correct ones and calculate the classification accuracy.

**Task.** Why is this accuracy close to 0.1?

Associate the model with an optimizer, a loss function and a metric like accuracy. Here we use the simplest stochastic gradient descent as the optimizer:

In [ ]:
model.compile('sgd',
              'sparse_categorical_crossentropy',
              'accuracy')

Once the model is associated with a loss function and a metric, calculate their values on any dataset by calling a built-in method:

In [ ]:
model.evaluate(DATA0, TARGET0)
model.evaluate(DATA1, TARGET1)

313/313 [==============================] - 1s 3ms/step - loss: 2.3189 - accuracy: 0.1176


[2.3188958168029785, 0.11760000139474869]

The accuracy on the validation dataset is indeed equal to what we calculated manually. Now train the model on the training dataset. Do only a few epochs for now:

In [ ]:
model.fit(DATA0, TARGET0, epochs = 10)

Epoch 1/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.7690 - accuracy: 0.8211
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4540 - accuracy: 0.8825
Epoch 3/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4017 - accuracy: 0.8928
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3755 - accuracy: 0.8978
Epoch 5/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3591 - accuracy: 0.9013
Epoch 6/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3474 - accuracy: 0.9036
Epoch 7/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3383 - accuracy: 0.9060
Epoch 8/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3314 - accuracy: 0.9079
Epoch 9/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3255 - accuracy: 0.9094
Epoch 10/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.320

The built-in training loop displays the loss and accuracy on the training dataset after each epoch. It also prints some other information.

**Task.** What does the progress bar indicate?  What does the number of 1 875 stand for?

The accuracy on the training dataset may not be representative for new samples due to the so-called overfitting. To get a more reliable estimation of the model accuracy, calculate it on the validation dataset. Also change the batch size and do more epochs to perform a full training:

In [ ]:
model.fit(DATA0, TARGET0, batch_size = 100, epochs = 100, validation_data = (DATA1, TARGET1))

Epoch 1/100
600/600 [==============================] - 2s 3ms/step - loss: 0.3169 - accuracy: 0.9120 - val_loss: 0.3044 - val_accuracy: 0.9168
Epoch 2/100
600/600 [==============================] - 2s 3ms/step - loss: 0.3157 - accuracy: 0.9122 - val_loss: 0.3033 - val_accuracy: 0.9164
Epoch 3/100
600/600 [==============================] - 2s 3ms/step - loss: 0.3145 - accuracy: 0.9126 - val_loss: 0.3027 - val_accuracy: 0.9165
Epoch 4/100
600/600 [==============================] - 2s 3ms/step - loss: 0.3132 - accuracy: 0.9129 - val_loss: 0.3017 - val_accuracy: 0.9159
Epoch 5/100
600/600 [==============================] - 2s 3ms/step - loss: 0.3122 - accuracy: 0.9132 - val_loss: 0.3007 - val_accuracy: 0.9161
Epoch 6/100
600/600 [==============================] - 2s 3ms/step - loss: 0.3111 - accuracy: 0.9139 - val_loss: 0.3002 - val_accuracy: 0.9171
Epoch 7/100
600/600 [==============================] - 2s 3ms/step - loss: 0.3101 - accuracy: 0.9140 - val_loss: 0.2989 - val_accuracy: 0.9168

**Task.** With the batch of 100, a single epoch executes roughly twice faster than with the default batch of 32. Why? What may be the other consequences of changing the batch size from 32 to 100?

Although this is not absolutely necessary, evaluate the model on the training and validation dataset:

In [ ]:
model.evaluate(DATA0, TARGET0)
model.evaluate(DATA1, TARGET1)

313/313 [==============================] - 1s 3ms/step - loss: 0.2744 - accuracy: 0.9224


[0.27443674206733704, 0.9223999977111816]

**Task.** The validation accuracy displayed after the last training epoch is equal to the validation accuracy evaluated after completing the training. This is not the case for the accuracy on the training dataset. Why?

In the following, by the validation accuracy of a model, we will always mean the maximum value listed during training, even if it is not in the last epoch. This is because Tensorflow can stop training after any epoch and thus produce a model with validation accuracy corresponding to that epoch. Now display a summary of the model architecture:

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


**Task.** Why does the number of weights equal 7 850?

**Task.** In this example, the training and validation accuracies are very close to each other, which means that there is almost no overfitting. Why?

In order to increase the overall accuracy, add a second layer. The number of neurons in that layer may be, for instance, the geometric mean of the number of inputs and outputs, that is, sqrt(10 * 784) = 89, or the next power of two, that is, 128. Use the classical sigmoid activation for this layer:

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation = 'sigmoid'),
    tf.keras.layers.Dense(10, activation = 'softmax')])

model.compile('sgd',
              'sparse_categorical_crossentropy',
              'accuracy')

model.fit(DATA0, TARGET0, batch_size = 100, epochs = 100, validation_data = (DATA1, TARGET1))

model.summary()

Epoch 1/100
600/600 [==============================] - 2s 3ms/step - loss: 1.9791 - accuracy: 0.5615 - val_loss: 1.6498 - val_accuracy: 0.7549
Epoch 2/100
600/600 [==============================] - 2s 3ms/step - loss: 1.4040 - accuracy: 0.7703 - val_loss: 1.1588 - val_accuracy: 0.8092
Epoch 3/100
600/600 [==============================] - 2s 3ms/step - loss: 1.0275 - accuracy: 0.8112 - val_loss: 0.8809 - val_accuracy: 0.8315
Epoch 4/100
600/600 [==============================] - 2s 3ms/step - loss: 0.8195 - accuracy: 0.8343 - val_loss: 0.7269 - val_accuracy: 0.8520
Epoch 5/100
600/600 [==============================] - 2s 3ms/step - loss: 0.6985 - accuracy: 0.8486 - val_loss: 0.6331 - val_accuracy: 0.8647
Epoch 6/100
600/600 [==============================] - 2s 3ms/step - loss: 0.6208 - accuracy: 0.8586 - val_loss: 0.5694 - val_accuracy: 0.8691
Epoch 7/100
600/600 [==============================] - 2s 3ms/step - loss: 0.5667 - accuracy: 0.8659 - val_loss: 0.5239 - val_accuracy: 0.8783

**Task.** Explain the number of weights in this network.

Adding the second layer improved the accuracy surprisingly little. Change the activation function of the added layer to relu, which is one of the most used:

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(10, activation = 'softmax')])

model.compile('sgd',
              'sparse_categorical_crossentropy',
              'accuracy')

model.fit(DATA0, TARGET0, batch_size = 100, epochs = 100, validation_data = (DATA1, TARGET1))

model.summary()

Epoch 1/100
600/600 [==============================] - 2s 3ms/step - loss: 1.0572 - accuracy: 0.7561 - val_loss: 0.5544 - val_accuracy: 0.8716
Epoch 2/100
600/600 [==============================] - 2s 3ms/step - loss: 0.4851 - accuracy: 0.8784 - val_loss: 0.4069 - val_accuracy: 0.8948
Epoch 3/100
600/600 [==============================] - 2s 3ms/step - loss: 0.3955 - accuracy: 0.8938 - val_loss: 0.3555 - val_accuracy: 0.9032
Epoch 4/100
600/600 [==============================] - 2s 3ms/step - loss: 0.3549 - accuracy: 0.9019 - val_loss: 0.3282 - val_accuracy: 0.9096
Epoch 5/100
600/600 [==============================] - 2s 3ms/step - loss: 0.3299 - accuracy: 0.9081 - val_loss: 0.3090 - val_accuracy: 0.9133
Epoch 6/100
600/600 [==============================] - 2s 3ms/step - loss: 0.3115 - accuracy: 0.9124 - val_loss: 0.2924 - val_accuracy: 0.9185
Epoch 7/100
600/600 [==============================] - 2s 3ms/step - loss: 0.2968 - accuracy: 0.9164 - val_loss: 0.2808 - val_accuracy: 0.9219

Now the improvement is very significant. The difference is because sigmoid saturates on both sides, yields vanishing gradients there, and effectively stucks the minimization. Relu does not saturate on the positive side and causes no such problems.

**Task.** The training takes less physical time with the relu activation. Why?

Now switch the optimizer to Adam, which is one of the best and most used:

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(10, activation = 'softmax')])

model.compile('adam',
              'sparse_categorical_crossentropy',
              'accuracy')

model.fit(DATA0, TARGET0, batch_size = 100, epochs = 100, validation_data = (DATA1, TARGET1))

model.summary()

Epoch 1/100
600/600 [==============================] - 2s 3ms/step - loss: 0.3331 - accuracy: 0.9088 - val_loss: 0.1761 - val_accuracy: 0.9493
Epoch 2/100
600/600 [==============================] - 2s 3ms/step - loss: 0.1514 - accuracy: 0.9565 - val_loss: 0.1310 - val_accuracy: 0.9613
Epoch 3/100
600/600 [==============================] - 2s 3ms/step - loss: 0.1086 - accuracy: 0.9690 - val_loss: 0.1084 - val_accuracy: 0.9670
Epoch 4/100
600/600 [==============================] - 2s 3ms/step - loss: 0.0846 - accuracy: 0.9752 - val_loss: 0.0904 - val_accuracy: 0.9727
Epoch 5/100
600/600 [==============================] - 2s 3ms/step - loss: 0.0683 - accuracy: 0.9802 - val_loss: 0.0836 - val_accuracy: 0.9744
Epoch 6/100
600/600 [==============================] - 2s 3ms/step - loss: 0.0555 - accuracy: 0.9837 - val_loss: 0.0804 - val_accuracy: 0.9754
Epoch 7/100
600/600 [==============================] - 2s 3ms/step - loss: 0.0461 - accuracy: 0.9864 - val_loss: 0.0725 - val_accuracy: 0.9779

The final validation accuracy is only slightly higher but Adam achieved it in a much smaller number of epochs than SGD. From now on, we will always use the Adam optimizer and the relu activation function. It is impressive that such a simple two-layer network correctly classifies 100% of the training dataset. This means that the lower accuracy on the validation dataset is uniquely due to overfitting.

**Task.** Why is there a significant overfitting here?

**Task.** Since there is overfitting caused by the already large number of model weights, it is dubious whether adding a third layer would still improve things, because it will increase the number of weights even more. Check this by adding a third layer. Adjust the layer sizes, the batch size, and the number of epochs to obtain a maximum validation accuracy.

Adding the third layer improved the validation accuracy only slightly. But this is still an achievement because the closer we are to 100%, the more difficult it is to gain accuracy. Note that even a slight increase in accuracy leads to a significant reduction in the error rate, which is complementary to accuracy. Here for instance, increasing the accuracy from 0.980 to 0.985 reduces the error rate from 2% to 1.5%, that is, by a factor of 4/3.

**Task.** Check that adding a fourth dense layer does not further increase the validation accuracy.

Since adding more dense layers does not further increase the validation accuracy, we must resort to other means. There are dedicated techniques to reduce overfitting, but it seems more interesting to introduce convolutional layers. First, load the data once again:

In [ ]:
(DATA0, TARGET0), (DATA1, TARGET1) = tf.keras.datasets.mnist.load_data()

Convolutional layers require the data to be four-dimensional, with the subsequent dimensions being the batch size, the image height, the image width, and the number of channels. The number of channels equals three for RGB images and one for grayscale images. We deal with grayscale images but still the fourth dimension needs to be added with the size of one:

In [ ]:
DATA0 = DATA0[:, :, :, None].astype('float32') / 255.
DATA1 = DATA1[:, :, :, None].astype('float32') / 255.

Construct a convolutional layer, pass some data through it, convert the output to a numpy array, and print its shape:

In [ ]:
layer = tf.keras.layers.Conv2D(8, 5, activation = 'relu')

OUTPUT1 = layer(DATA1).numpy()
print(OUTPUT1.shape)

(10000, 24, 24, 8)


This sample convolutional layer has 8 output channels, which means that it produces an output with 8 channels. It also has a filter of size 5, which means that every output pixel is a linear combination of input pixels in a 5x5 square. Consequenntly, the layer reduces the image size by 4 in each dimension, from 28 to 24 in this case. In order to perform image classification, the output from the convolutional layer must be flattened and passed to a dense layer with softmax activation:

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(8, 5, activation = 'relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation = 'softmax')])

model.compile('adam',
              'sparse_categorical_crossentropy',
              'accuracy')

model.fit(DATA0, TARGET0, batch_size = 100, epochs = 100, validation_data = (DATA1, TARGET1))

model.summary()

Epoch 1/100
600/600 [==============================] - 5s 4ms/step - loss: 0.3291 - accuracy: 0.9089 - val_loss: 0.1681 - val_accuracy: 0.9518
Epoch 2/100
600/600 [==============================] - 2s 4ms/step - loss: 0.1260 - accuracy: 0.9654 - val_loss: 0.0887 - val_accuracy: 0.9726
Epoch 3/100
600/600 [==============================] - 2s 4ms/step - loss: 0.0801 - accuracy: 0.9775 - val_loss: 0.0669 - val_accuracy: 0.9796
Epoch 4/100
600/600 [==============================] - 2s 4ms/step - loss: 0.0616 - accuracy: 0.9820 - val_loss: 0.0642 - val_accuracy: 0.9796
Epoch 5/100
600/600 [==============================] - 3s 4ms/step - loss: 0.0516 - accuracy: 0.9851 - val_loss: 0.0573 - val_accuracy: 0.9826
Epoch 6/100
600/600 [==============================] - 2s 4ms/step - loss: 0.0451 - accuracy: 0.9864 - val_loss: 0.0586 - val_accuracy: 0.9822
Epoch 7/100
600/600 [==============================] - 3s 4ms/step - loss: 0.0385 - accuracy: 0.9886 - val_loss: 0.0587 - val_accuracy: 0.9827

**Task.** Why does the covolutional layer have 208 weights?

**Task.** Why does the total number of weights equal 46 298?

This two-layer model with 46 298 weights is almost as good the three-layer dense model with 218 058 weights! This is partly because convolutional layers have very little weights, so they are less prone to overfitting than dense layers. Convolutional networks frequently use pooling layers that reduce the spatial image size twice in each direction by retaining only the pixel with maximum activation from each 2x2 square. Pooling acts independently in each channel. Add pooling after the convolutional layer:

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(8, 5, activation = 'relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation = 'softmax')])

model.compile('adam',
              'sparse_categorical_crossentropy',
              'accuracy')

model.fit(DATA0, TARGET0, batch_size = 100, epochs = 100, validation_data = (DATA1, TARGET1))

model.summary()

Epoch 1/100
600/600 [==============================] - 3s 4ms/step - loss: 0.3959 - accuracy: 0.8923 - val_loss: 0.1729 - val_accuracy: 0.9520
Epoch 2/100
600/600 [==============================] - 2s 4ms/step - loss: 0.1500 - accuracy: 0.9573 - val_loss: 0.1149 - val_accuracy: 0.9670
Epoch 3/100
600/600 [==============================] - 2s 4ms/step - loss: 0.1097 - accuracy: 0.9686 - val_loss: 0.0889 - val_accuracy: 0.9738
Epoch 4/100
600/600 [==============================] - 2s 4ms/step - loss: 0.0885 - accuracy: 0.9747 - val_loss: 0.0766 - val_accuracy: 0.9764
Epoch 5/100
600/600 [==============================] - 2s 4ms/step - loss: 0.0753 - accuracy: 0.9785 - val_loss: 0.0716 - val_accuracy: 0.9779
Epoch 6/100
600/600 [==============================] - 2s 4ms/step - loss: 0.0662 - accuracy: 0.9806 - val_loss: 0.0636 - val_accuracy: 0.9783
Epoch 7/100
600/600 [==============================] - 2s 4ms/step - loss: 0.0596 - accuracy: 0.9819 - val_loss: 0.0595 - val_accuracy: 0.9804

**Task.** Examine the total number of weights in this network.

Pooling inevitably looses some information, so the current network has worse validation accuracy, but only a bit worse. Convolutional networks are often built of pairs of convolutional and pooling layers. Add another such pair:

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(8, 5, activation = 'relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(16, 5, activation = 'relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation = 'softmax')])

model.compile('adam',
              'sparse_categorical_crossentropy',
              'accuracy')

model.fit(DATA0, TARGET0, batch_size = 100, epochs = 100, validation_data = (DATA1, TARGET1))

model.summary()

Epoch 1/100
600/600 [==============================] - 4s 5ms/step - loss: 0.3673 - accuracy: 0.8940 - val_loss: 0.1421 - val_accuracy: 0.9596
Epoch 2/100
600/600 [==============================] - 3s 5ms/step - loss: 0.1247 - accuracy: 0.9629 - val_loss: 0.0912 - val_accuracy: 0.9736
Epoch 3/100
600/600 [==============================] - 3s 5ms/step - loss: 0.0924 - accuracy: 0.9723 - val_loss: 0.0725 - val_accuracy: 0.9773
Epoch 4/100
600/600 [==============================] - 3s 5ms/step - loss: 0.0745 - accuracy: 0.9770 - val_loss: 0.0591 - val_accuracy: 0.9811
Epoch 5/100
600/600 [==============================] - 3s 5ms/step - loss: 0.0631 - accuracy: 0.9808 - val_loss: 0.0488 - val_accuracy: 0.9849
Epoch 6/100
600/600 [==============================] - 3s 5ms/step - loss: 0.0558 - accuracy: 0.9833 - val_loss: 0.0463 - val_accuracy: 0.9849
Epoch 7/100
600/600 [==============================] - 3s 5ms/step - loss: 0.0492 - accuracy: 0.9850 - val_loss: 0.0504 - val_accuracy: 0.9829

**Task.** How to explain the 3 216 weights in the second convolutional layer?

Each pair of convolutional and pooling layers reduces the spatial image size more than four times. It is a common practice to inrease the number of output channels twice in each subsequent convolutional layer, as we did. In this way, each pair reduces the number of features roughly twice. This in turn significantly reduces the number of weights in the final dense layer and in the entire net, thus making it less and less overfitting. This network has less weights than any previous one, and yet this is the first time that the validation accuracy exceeds 99%.

**Task.** Add a third pair of convolutional and pooling layers. What number of output channels and what filter size should be used in the third convolution?

The validation accuracy is slightly higher than with two pairs of convolutional and pooling layers, but there is a better way to improve this network.

**Task.** In the case of dense networks, three layers was the optimal number. It seems that with convolutional networks two pairs of convolutional and pooling layers are a good solution. How to combine these observations?

**Task.** Try other architectures, vary the number of layers, the number of channels, the filter sizes etc., to get an even better accuracy on the validation dataset.